In [24]:
# https://jovianlin.io/embeddings-in-keras/
# 참고사이트
import re
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Bidirectional, LSTM
docs = ['Well done!', 'Good work', 'Great effort', 'nice work', 'Excellent!',
        'Weak', 'Poor effort!', 'not good', 'poor work', 'Could have done better.']

labels = np.array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [37]:
own_embedding_vocab_size = 10
encoded_docs_oe = [one_hot(d, own_embedding_vocab_size) for d in docs]
print(encoded_docs_oe)
print(len(encoded_docs_oe))

[[5, 3], [3, 9], [9, 4], [7, 9], [7], [5], [6, 4], [7, 3], [6, 9], [3, 8, 3, 2]]
10


In [39]:
maxlen = 5
padded_decs_oe = pad_sequences(encoded_docs_oe, maxlen=maxlen, padding='post')
print(padded_decs_oe)
print(padded_decs_oe.shape)

[[5 3 0 0 0]
 [3 9 0 0 0]
 [9 4 0 0 0]
 [7 9 0 0 0]
 [7 0 0 0 0]
 [5 0 0 0 0]
 [6 4 0 0 0]
 [7 3 0 0 0]
 [6 9 0 0 0]
 [3 8 3 2 0]]
(10, 5)


In [27]:
model = Sequential()
model.add(Embedding(input_dim=own_embedding_vocab_size, output_dim=32, input_length=maxlen))
model.add(Bidirectional(LSTM(32,activation='relu',recurrent_dropout=0.1,return_sequences)))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

In [29]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])  # Compile the model
print(model.summary())  # Summarize the model

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 32)             320       
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 17,025
Trainable params: 17,025
Non-trainable params: 0
_________________________________________________________________
None


In [40]:
model.fit(padded_decs_oe, labels, epochs=50, verbose=0)  # Fit the model
loss, accuracy = model.evaluate(padded_decs_oe, labels, verbose=0)  # Evaluate the model
print('loss : %0.3f'%loss ,'Accuracy: %0.3f' % accuracy)

loss : 0.153 Accuracy: 0.900


In [31]:
padded_decs_oe[1:2]

array([[3, 9, 0, 0, 0]])

In [42]:
word = "Good work"
word_docs_oe = [one_hot(word, own_embedding_vocab_size)]
word_oe = pad_sequences(word_docs_oe, maxlen=maxlen, padding='post')
pred = model.predict(word_oe)
np.round(pred)

array([[1.]], dtype=float32)

In [43]:
s = padded_decs_oe
a = model.predict(s)
np.round(a)

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [33]:
word_oe

array([[3, 9, 0, 0, 0]])

In [34]:
s = padded_decs_oe
a = model.predict(s)
a

array([[0.25155085],
       [0.88942003],
       [0.9993516 ],
       [0.9988058 ],
       [0.76390016],
       [0.39815846],
       [0.0150933 ],
       [0.34821892],
       [0.0957509 ],
       [0.00982311]], dtype=float32)

In [35]:
np.round(a)

array([[0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)